In [99]:
from requests import Request, Session
import pandas as pd
import json
import time
import webbrowser
import pprint
import gspread
import csv
from datetime import date, timedelta
from dateutil import parser

headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '5af15786-87c3-45f8-92de-214a92592503'
}

session = Session()
session.headers.update(headers)

today = date.today()
d_min = date.today() - timedelta(days=365)

## GET TOP 10 EXCHANGE LISTS

In [100]:
url_map_id = 'https://pro-api.coinmarketcap.com/v1/exchange/map'
parameters_map_id = { 'start' : 1,
               'limit' : 10,
               'sort' : 'volume_24h',
               }
response_map_id = session.get(url_map_id, params = parameters_map_id)
top_xc = pd.json_normalize(json.loads(response_map_id.text)['data'])
top_xc = top_xc[['id','name','slug']]
top_xc

,id,name,slug
0,270,Binance,binance
1,294,OKX,okx
2,1064,BingX,bingx
3,544,MEXC,mxc
4,1182,Deepcoin,deepcoin
5,1563,BTCEX,btcex-exchange
6,521,Bybit,bybit
7,525,XT.COM,xt
8,524,FTX,ftx
9,517,CoinW,coinw


url = 'https://pro-api.coinmarketcap.com/v1/exchange/market-pairs/latest'

c = pd.DataFrame()
for i in top_xc['slug']:
    parameters = { 'slug' : i,
               'start' : 1,
               'limit' : 10,
               'category' : 'spot'
               }
    response = session.get(url, params=parameters)
    top_xc1 = pd.json_normalize(json.loads(response.text))
    c = pd.concat([c,top_xc1])

c

## GET EXCHANGE VOLUME BY DATE

In [101]:
url_historical = 'https://pro-api.coinmarketcap.com/v1/exchange/quotes/historical'

list_xc = list(top_xc['slug'])
slug_xc = ', '.join(list_xc).replace(' ', '')
parameters_historical = { 'slug': slug_xc,
                          'interval' : 'daily',
                          'time_start' : d_min,
                          'time_end' : today}

response_historical = session.get(url_historical, params = parameters_historical)
df = pd.DataFrame()

for i in list_xc:
    a = pd.json_normalize(json.loads(response_historical.text)['data'][i]['quotes'])
    a['exchange'] = i
    a = a.rename(columns= {'quote.USD.volume_24h':'usd_vol24h'})
    a = a[['timestamp',
           'exchange',
           'usd_vol24h']]
    df = pd.concat([df,a])

df

,timestamp,exchange,usd_vol24h
0,2021-09-14T00:00:16.000Z,binance,1.249449e+11
1,2021-09-15T00:00:16.000Z,binance,8.257044e+10
2,2021-09-16T00:00:16.000Z,binance,7.996439e+10
3,2021-09-17T00:00:14.000Z,binance,8.988862e+10
4,2021-09-18T00:00:14.000Z,binance,7.840808e+10
...,...,...,...
360,2022-09-09T00:00:11.000Z,coinw,-1.000000e+00
361,2022-09-10T00:00:11.000Z,coinw,-1.000000e+00
362,2022-09-11T00:00:11.000Z,coinw,-1.000000e+00
363,2022-09-12T00:00:11.000Z,coinw,-1.000000e+00


In [102]:
df.to_csv('/Users/ghazian/Downloads/historical_volume.csv')